In [ ]:
#!pip3 install pytelegrambotapi
import telebot
from bs4 import BeautifulSoup
import requests

#скрипт по созданию телеграм бота,который на сообщение с городом отправляет текущую погоду
#данные используються с сайта https://pogoda.mail.ru/
#для работы бота требуется указать правильное название города строчными буквами. Например для zurich нужно указать tsurikh,вместо стандартного moscow нужно указать moskva
#наблюдаются некоторые погрешности с полнотой вывода информации 

#наши глобальные переменные
api_key = '' #ваш апи бота которые получаете через fatherbota
url = 'https://pogoda.mail.ru/prognoz/'

#функция получает html код нашего url
def get_html(url):
    r = requests.get(url)
    return r.text 

#функция обрабытвает наш html код и вытаскивает нужные данные при этом обрабатываем на переносы,пробелы,лишние символы
def get_page_data (html):
  soup = BeautifulSoup(html,'lxml')
  div = soup.find('div',class_='wrapper').find('div',class_='information block js-city_one').find('div',class_='information__content').find('div',class_='information__content__wrapper information__content__wrapper_left').text
  html_string=[]
  for ele in str(div).split("\n"):
    html_string.append(ele.strip())
    html_string = list(filter(None, html_string))
    s = str(html_string ).strip('[]')
    s= s.replace("'", "")
    s = s[:-3]
  return s


#подключаемся к боту,указываем команду и ответ на нее
bot = telebot.TeleBot(api_key)
@bot.message_handler(commands=['go'])
def hello(message):
   bot.send_message(message.chat.id,'Hi что бы узнать текущую погоду введите название города в формате "bucharest"')


#получаем ответ пользователя и возращаем результат при этом использую проверку на строчные буквы
@bot.message_handler(content_types=['text'])
def weather(message):
  if message.text.lower() == message.text:
    bot.send_message(message.chat.id,get_page_data(get_html(url+message.text)))
  else:
    bot.send_message(message.chat.id,'указано же.. город в формате "bucharest"')

#запуск бота в работу
bot.polling()
